<a href="https://colab.research.google.com/github/qut-dmrc/fbadsearch/blob/main/Facebook_ad_api_collector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First, set up authorisation

In [ ]:
from google.colab import auth
auth.authenticate_user()

# fetch the python api wrapper from github
!pip3 install -e git+https://github.com/qut-dmrc/fbadsearch.git#egg=fbadsearch

# Restart the runtime before proceeding, and run the first cell again.

In [ ]:
# restart the runtime before this step here
from fbadsearch import FacebookAdSearch, save_data, fetch_ads, load_to_bq

In [ ]:
#@title If you already have a long-lived token, enter it here:
LONG_LIVED_ACCESS_TOKEN = '' #@param {type: 'string'}

In [ ]:
#@title Otherwise, get your access token here https://developers.facebook.com/tools/explorer/297028494528207/
ACCESS_TOKEN = ''  #@param {type: 'string'}


In [ ]:
#@title GCS path and BigQuery table to save to:
GC_PROJECT = '' #@param {type: 'string'}
GCS_BUCKET = ''#@param {type: 'string'}
GCS_DIR = "" #@param {type: 'string'}
BQ_TABLE = '' #@param {type: 'string'}

import datetime
timestamp = datetime.datetime.utcnow().isoformat()
GCS_DIR = f'{GCS_DIR}/{timestamp}'

In [ ]:
if not LONG_LIVED_ACCESS_TOKEN:
  # Get long-lived token
  url = "https://graph.facebook.com/v9.0/oauth/access_token"
  params = {'grant_type': 'fb_exchange_token',          
            'client_id': 'YOUR_CLIENT_ID',
            'client_secret': 'YOUR_CLIENT_SECRET',
            'fb_exchange_token': ACCESS_TOKEN}

  r = requests.get(url, params=params)
  data = r.json()

  LONG_LIVED_ACCESS_TOKEN = data['access_token']
  print(f'Got long lived token:\n{LONG_LIVED_ACCESS_TOKEN}')
else:
  print(f'Using long-lived token provided.')

In [ ]:
import logging

log_file = f'ad_collector_{timestamp}.log'
log_formatter = "%(asctime)s [%(filename)-20.20s:%(lineno)-4.4s - %(funcName)-20.20s() [%(levelname)-8.8s]  %(message).5000s"

logging.basicConfig(level=logging.DEBUG, filename=log_file, filemode='a', format=log_formatter)

# Create a custom logger
logger = logging.getLogger('fbadsearch')

# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)

# Create formatters and add it to handlers
c_format = logging.Formatter(log_formatter)
c_handler.setFormatter(c_format)

# Add handlers to the logger
logger.addHandler(c_handler)

logger.info(f"Logging to {log_file}")

# Configuration

In [ ]:
# NOTE: I replaced 'AmandaCamm4Senate' with page ID '832302137132814' for https://www.facebook.com/AmandaCammMP/

# this is all page IDs from '141020_Qld_Election_Final Candidates_FB_Twitter.xlsx', which was derived from the electoral roll
pol_page_ids = [785196888157078, 1785579221658334, 106607231221280, 100506221799325, 104524101187625, 105553407983008, 109518917420445, 103650197726925, 
                909423379175033, 1693010970976516, 1832562330303269, 1777080942596298, 109959002750968, 407720832712714, 2220634441369101, 338368899845370, 
                117656986742891, 100636498106021, 567004083977539, 114001797119430, 104299284722230, 350660715317781, 112908010419196, 104221031461259, 
                100156858417093, 389125434502747, 103035638207098, 106380891185520, 115406840293372, 523586527672529, 736734193067726, 114150970305956, 
                103396634852846, 108650754237728, 116501956844030, 576823859628727, 107301347688602, 105913364458834, 263665090825710, 255774254930728, 
                101457271423792, 104106704739897, 705975076183979, 101709431690846, 1514961152071432, 1897562910520016, 657663438179343, 545743365504124,
                336275459877992, 1534219783464358, 100164035048916, 104845828020597, 102803318250426, 110055630739867, 1915186665431831, 641418173146075, 
                101229475097691, 1729147630711565, 115495863575291, 539690296167013, 112370233821131, 114523806906481, 467602216674884, 113204303738366, 
                113313763488937, 109449280530584, 111225464067837, 108467557636953, 103481191448837, 1863325200581981, 105224701294391, 113474843753954, 
                395201170875812, 108660834188644, 1256779447739574, 114200597031740, 289973794486663, 461093597424764, 1167376913283953, 1509842982624646, 
                292763430848105, 142712312762123, 111003630759377, 106556794289046, 104761038015826, 100136771873274, 432474437121377, 107231951131409, 
                102121481379494, 1169942783020880, 110725204092217, 249840622106964, 1725156551117500, 520765594773519, 263462127087524, 1208758175964023, 
                104078437826807, 113096467226142, 100166321803017, 109251870932868, 101417321688609, 101185374929025, 1410167955931527, 567814099965388,
                213302842097486, 105893381227684, 113275300532255, 227843448632578, 119529791992309, 149428025406938, 106479674497366, 105883917945171,
                113209043869509, 115362773499528, 221723521179531, 581527922536548, 100100165143301, 233777047885666, 121738193028105, 109800067441452, 
                106101341249183, 106403251214496, 111124140659278, 110966290591055, 207989436076755, 112450560339784, 576766316323347, 102096831555854, 
                328287840654244, 109343407457327, 111903673975158, 111707477258429, 112793727216427, 106725931187761, 100996221715253, 107765634022348, 
                101731018226021, 106213717906020, 1196449963861543, 587118181959376, 352801041811373, 100568588460862, 102429501580044, 324660077589275, 
                105624107957969, 106307547865969, 102488524573456, 606278869581152, 103635297783684, 283965465110918, 363184810447890, 106496577802582, 
                1675215042771280, 513154475468041, 1598711303739855, 100190838538821, 109943174169252, 231818983678379, 101335338377069, 101383795020479,
                709617325724631, 113230193550827, 775462745806585, 1845950472089687, 107410651126691, 106440287879664, 554268431430489, 740083362711653, 
                788385304507966, 100499011732656, 303244313358772, 615734159083360, 229474257118871, 105098314587657, 103072239832731, 103109601504421, 
                108155947685109, 1524300257809805, 114108316662776, 113642363810676, 111647547204231, 101015264723836, 113153533805269, 2055206548040133,
                126948162124511, 111334597214496, 1568113776764144, 656903121164867, 109853183890303, 110344233985845, 113957616692644, 1985122728369782,
                1453598948193351, 423996701839665, 101351228230215, 101470824951637, 103683874718454, 168475823483405, 111556703767159, 102398231198500,
                608640325907244, 686500954867746, 836025246434769, 107664064169906, 103403601543186, 102092101394038, 101149561446436, 109126144006684,
                101906174920141, 774386052738210, 220677281357963, 297511687070414, 103086624806616, 107008534510908, 522394997927024, 102385998294579, 
                104967071007331, 1374427989490497, 105372087602238, 101728244995980, 113030643782486, 112793877175216, 107376851136972, 236444377561800,
                106674377818894, 1408935539376139, 474104612632953, 1657049431237380, 233782593477030, 476400975832131, 159541697531443, 2338085742938967, 
                624349244885583, 113581900482266, 1430189740388666, 121967566067770, 203933633277603, 554347721374792, 102145868266091, 847722658587647,
                417714538385668, 112436110439247, 1134296200015702, 1492121741050481, 344226585660387, 396892697109784, 101046031735037, 110315050799043,
                612853245438547, 100614331807415, 104192634278, 1448950485153928, 108865297506178, 1233955626642726, 1496534207028271, 106194461192104,
                100402335145235, 861586993882334, 116058153567277, 1910997169220735, 117245719917565, 2231610370442955, 247062522139479, 1804660016424957, 
                1718173078398192, 1563990510511621, 1954574018107631, 101355847935667, 1114525825238202, 100986351788907, 982578781943135, 114095057091976, 
                108724607614834, 484472008399989, 114859093693324, 760212544116382, 106936114474003, 394508164021609, 109558924169959, 576746423001703,
                254204537980710, 105778420979705, 100752448317793, 101221321379731, 101227101730425, 108330807693201, 850215955156264, 220443132720971,
                109115337563838, 113281133715746, 110039737541952, 1325182927495629, 102316024969687, 590171327666970, 101511358342349, 102047474939736,
                451516984861373, 372985473548850, 105352941226837, 109312463945087, 104042988033763, 100235271790601, 900067690141232, 100161275099274,
                355838727797763, 543159962822554, 101560051565626, 104776624407280, 504738263066099, 117236096581797, 100207568515359, 295613770481116,
                1856952217890753, 632027813515639, 116553630206367, 113620733840833, 100410168353799, 1533558346882013, 126980844673171, 106181434300985,
                105856501251723, 662955787059972, 180764285343226, 516047178492348, 2344361252476442, 135521496939589, 109879534099888, 116716883502488,
                103834321490119, 583679722297960, 1818359645047406, 109032010961656, 212140395561960, 102553308232651, 123650765790366, 373143696085132,
                435436303151180, 1524698857748303, 111245177414703, 286377921487406, 110889697057629, 101071261582099, 1408801042745655, 101994945016870,
                1947325512179992, 223247229116486, 106812531111470, 109056410952157, 109656674159830, 104677414717000, 104532491409741, 584965482184059,
                100537861830777, 113279090531389, 602894639867090, 418860658142824, 131130543754756, 112750263934699, 104364204671011, 101606898126747,
                109755660886785, 103124224780415, 742367169159127, 103622581416252, 110232210680243, 331007484332138, 1296089260454472, 100689074952671,
                104764508061113, 102018878207644, 113555240489914, 224778514198891, 191008468082204, 124454401575649, 210143127101006, 133032931544819,
                102151004917932, 104804378054435, 101444028195549, 395174637282859, 109759564122839, 832302137132814]

pol_page_ids = [ str(id) for id in pol_page_ids]              

In [ ]:
config_text = """
ad_active_status: ALL 
ad_type: POLITICAL_AND_ISSUE_ADS
ad_reached_countries: 
  - AU
# Fields for querying all possible metadata from the API
query_fields:
  - ad_creation_time
  - ad_creative_body
  - ad_creative_link_caption
  - ad_creative_link_description
  - ad_creative_link_title
  - ad_delivery_start_time
  - ad_delivery_stop_time
  - ad_snapshot_url
  - demographic_distribution
  - funding_entity
  - impressions
  - page_id
  - page_name
  - region_distribution
  - spend

start_date: 2020-08-01
end_date: 2020-10-31

"""

import yaml
from more_itertools import chunked

config = yaml.safe_load(config_text)

# Fetch all data
For each batch of ten page IDs, get all their ads for the timeperiod, save to Google Cloud Storage, and load to BigQuery

In [ ]:
from google.cloud import storage
gcs_client = storage.Client(project=GC_PROJECT)
fetch_and_save_ads(pol_page_ids, LONG_LIVED_ACCESS_TOKEN, config, gcs_client, GCS_BUCKET, GCS_DIR)        

In [ ]:
# upload the datafile to bigquery
from google.cloud import bigquery
bq_client = bigquery.Client(project=GC_PROJECT)

uri = f'gs://{GCS_BUCKET}/{GCS_DIR}/batch_*.json'

load_to_bq(bq_client, BQ_TABLE, uri)

# finally, save our log file

In [ ]:
# save log file
save_log_file(gcs_client, GCS_BUCKET, GCS_DIR, log_file)